In [3]:
import pandas as pd
import plotly.express as px
import dash
from dash import dcc, html
import dash_bootstrap_components as dbc
import requests
from io import BytesIO

# === Load data with requests (safe streaming) ===
url = "https://covid.ourworldindata.org/data/owid-covid-data.csv"
print("Downloading live dataset...")
response = requests.get(url)
df = pd.read_csv(BytesIO(response.content))
print("✅ Download complete.")

# === Filter and preprocess ===
df = df[df['continent'].notna()]
df = df[['location', 'date', 'total_cases', 'total_deaths', 'total_vaccinations',
         'population', 'continent', 'iso_code']]
df['total_cases'] = df['total_cases'].fillna(0)
df['total_deaths'] = df['total_deaths'].fillna(0)
df['total_vaccinations'] = df['total_vaccinations'].fillna(0)

# === Latest KPIs ===
latest_date = df['date'].max()
latest_df = df[df['date'] == latest_date]
total_cases = int(latest_df['total_cases'].sum())
total_deaths = int(latest_df['total_deaths'].sum())
total_vaccs = int(latest_df['total_vaccinations'].sum())

# === Dash App ===
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.FLATLY])
app.title = "🌐 Global COVID Dashboard"

app.layout = dbc.Container([
    html.H1("🌍 COVID-19 Global Tracker", className="text-center mb-4"),

    dbc.Row([
        dbc.Col(dbc.Card([
            dbc.CardBody([
                html.H5("Total Cases", className="card-title"),
                html.H3(f"{total_cases:,}", className="text-danger")
            ])
        ]), width=4),

        dbc.Col(dbc.Card([
            dbc.CardBody([
                html.H5("Total Deaths", className="card-title"),
                html.H3(f"{total_deaths:,}", className="text-dark")
            ])
        ]), width=4),

        dbc.Col(dbc.Card([
            dbc.CardBody([
                html.H5("Total Vaccinations", className="card-title"),
                html.H3(f"{total_vaccs:,}", className="text-success")
            ])
        ]), width=4),
    ], className="mb-4"),

    dbc.Row([
        dbc.Col([
            dcc.Graph(figure=px.bar(
                latest_df.sort_values(by='total_cases', ascending=False).head(10),
                x='total_cases', y='location', orientation='h',
                color='total_cases', color_continuous_scale='Reds',
                title="🏥 Top 10 Countries by Total Cases",
                labels={'location': 'Country', 'total_cases': 'Total Cases'}
            ).update_layout(yaxis=dict(autorange="reversed")))
        ], width=6),

        dbc.Col([
            dcc.Graph(figure=px.bar(
                latest_df.sort_values(by='total_deaths', ascending=False).head(10),
                x='total_deaths', y='location', orientation='h',
                color='total_deaths', color_continuous_scale='Blues',
                title="⚰️ Top 10 Countries by Total Deaths",
                labels={'location': 'Country', 'total_deaths': 'Total Deaths'}
            ).update_layout(yaxis=dict(autorange="reversed")))
        ], width=6),
    ]),

    html.Hr(),

    dbc.Row([
        dbc.Col([
            html.H4("🌐 Time-Lapse: COVID-19 Total Cases"),
            dcc.Graph(figure=px.choropleth(
                df,
                locations='iso_code',
                color='total_cases',
                hover_name='location',
                animation_frame='date',
                color_continuous_scale='Reds',
                range_color=(0, df['total_cases'].max()),
                title='🌍 Animated Map of COVID-19 Cases Over Time'
            ).update_geos(showframe=False, showcoastlines=False))
        ], width=12),
    ]),

    dbc.Row([
        dbc.Col([
            html.H4("💉 Time-Lapse: COVID-19 Total Vaccinations"),
            dcc.Graph(figure=px.choropleth(
                df,
                locations='iso_code',
                color='total_vaccinations',
                hover_name='location',
                animation_frame='date',
                color_continuous_scale='Greens',
                range_color=(0, df['total_vaccinations'].max()),
                title='🌍 Animated Map of COVID-19 Vaccinations Over Time'
            ).update_geos(showframe=False, showcoastlines=False))
        ], width=12),
    ])
], fluid=True)

# === Run server ===
if __name__ == "__main__":
    app.run(debug=True, port=8080)


✅ Download complete.
